In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
import requests
from datetime import datetime, timedelta
import geopandas as gpd
import folium
from tqdm import tqdm
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

class MLTransitAnalyzer:
    """A tool for analyzing public transit GTFS data with machine learning capabilities."""
    
    def __init__(self, data_dir="data"):
        """Initialize the analyzer with a data directory."""
        self.data_dir = data_dir
        self.models_dir = os.path.join(data_dir, "models")
        
        # Create directories
        os.makedirs(data_dir, exist_ok=True)
        os.makedirs(self.models_dir, exist_ok=True)
        
        # Storage for loaded data
        self.routes = None
        self.trips = None
        self.stop_times = None
        self.stops = None
        self.calendar = None
        
        # Storage for derived data
        self.route_stats = None
        self.stop_stats = None
        self.time_stats = None
        
        # Machine learning models
        self.demand_model = None
        self.demand_features = None
        self.scaler = None
        self.stop_clusters = None
        
    def download_gtfs(self, url):
        """Download GTFS data from a URL."""
        print(f"Downloading GTFS data from {url}...")
        
        # Download the zip file
        response = requests.get(url)
        zip_path = os.path.join(self.data_dir, "gtfs.zip")
        
        with open(zip_path, 'wb') as f:
            f.write(response.content)
        
        # Extract the files
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(self.data_dir)
        
        print("GTFS data downloaded and extracted successfully.")
    
    def load_data(self):
        """Load GTFS data from the data directory."""
        print("Loading GTFS data...")
        
        # Load the GTFS files
        try:
            self.routes = pd.read_csv(os.path.join(self.data_dir, "routes.txt"))
            self.trips = pd.read_csv(os.path.join(self.data_dir, "trips.txt"))
            self.stop_times = pd.read_csv(os.path.join(self.data_dir, "stop_times.txt"))
            self.stops = pd.read_csv(os.path.join(self.data_dir, "stops.txt"))
            self.calendar = pd.read_csv(os.path.join(self.data_dir, "calendar.txt"))
            
            print(f"Loaded {len(self.routes)} routes and {len(self.stops)} stops.")
            return True
        except Exception as e:
            print(f"Error loading data: {e}")
            return False
    
    def preprocess_data(self):
        """Preprocess the data for analysis and machine learning."""
        print("Preprocessing data...")
        
        if any(x is None for x in [self.routes, self.trips, self.stop_times, self.stops, self.calendar]):
            print("Data not loaded. Please load data first.")
            return False
        if self.routes is not None:
            self.routes['route_id'] = self.routes['route_id'].astype(str)
        if self.trips is not None:
            self.trips['route_id'] = self.trips['route_id'].astype(str)
            self.trips['trip_id'] = self.trips['trip_id'].astype(str)
        if self.stop_times is not None:
            self.stop_times['trip_id'] = self.stop_times['trip_id'].astype(str)
            self.stop_times['stop_id'] = self.stop_times['stop_id'].astype(str)
        if self.stops is not None:
            self.stops['stop_id'] = self.stops['stop_id'].astype(str)        
        
        # 1. Extract hour from departure_time
        def extract_hour(time_str):
            try:
                parts = time_str.split(':')
                hour = int(parts[0])
                if hour >= 24: 
                    hour = hour - 24
                return hour
            except:
                return None
        
        self.stop_times['departure_hour'] = self.stop_times['departure_time'].apply(extract_hour)
        
        # 2. Compute route statistics
        self.route_stats = self.analyze_route_frequency()
        
        # 3. Compute stop statistics
        self.stop_stats = self.analyze_stop_activity()
        
        # 4. Compute time statistics
        hourly_counts = self.stop_times.groupby('departure_hour').size()
        self.time_stats = pd.DataFrame({
            'hour': hourly_counts.index,
            'trip_count': hourly_counts.values
        })
        
        # 5. Join trips with calendar for day of week analysis
        trips_calendar = pd.merge(self.trips, self.calendar, on='service_id')
        
        # 6. Create a more detailed stop times dataset with trip and route info
        detailed_stop_times = pd.merge(
            self.stop_times,
            self.trips[['trip_id', 'route_id', 'service_id']],
            on='trip_id'
        )
        
        # 7. Add day of week indicators
        for day in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']:
            detailed_stop_times[day] = detailed_stop_times['service_id'].map(
                self.calendar.set_index('service_id')[day]
            )
        
        # 8. Create hourly demand by stop dataset for machine learning
        stop_hour_demand = detailed_stop_times.groupby(['stop_id', 'departure_hour']).size().reset_index(name='demand')
        
        # 9. Add stop location information
        stop_hour_demand = pd.merge(stop_hour_demand, self.stops[['stop_id', 'stop_lat', 'stop_lon']], on='stop_id')
        
        # 10. Add additional features for machine learning
        stop_hour_demand['is_rush_hour'] = ((stop_hour_demand['departure_hour'] >= 7) & 
                                            (stop_hour_demand['departure_hour'] <= 9) | 
                                            (stop_hour_demand['departure_hour'] >= 16) & 
                                            (stop_hour_demand['departure_hour'] <= 18)).astype(int)
        
        stop_hour_demand['is_business_hours'] = ((stop_hour_demand['departure_hour'] >= 9) & 
                                                (stop_hour_demand['departure_hour'] <= 17)).astype(int)
        
        stop_hour_demand['is_night'] = ((stop_hour_demand['departure_hour'] >= 22) | 
                                        (stop_hour_demand['departure_hour'] <= 5)).astype(int)
        
        # Store this dataset for machine learning
        self.demand_features = stop_hour_demand
        
        print("Data preprocessing complete.")
        return True
    

    def analyze_route_frequency(self):
        """Analyze the frequency of service by route."""
        if self.trips is None or self.routes is None:
            print("Data not loaded. Please load data first.")
            return None

        # Count trips per route
        trip_counts = self.trips.groupby('route_id').size().reset_index(name='trip_count')

        # Convert route_id to string in both dataframes to ensure type consistency
        trip_counts['route_id'] = trip_counts['route_id'].astype(str)
        self.routes['route_id'] = self.routes['route_id'].astype(str)

        # Merge with route information
        try:
            # Try to merge with route_long_name which is the standard field
            route_frequency = pd.merge(trip_counts, self.routes[['route_id', 'route_long_name']], on='route_id')
        except KeyError:
            # Some GTFS feeds use route_short_name instead of route_long_name
            print("Using route_short_name instead of route_long_name")
            if 'route_short_name' in self.routes.columns:
                route_frequency = pd.merge(trip_counts, self.routes[['route_id', 'route_short_name']], on='route_id')
                route_frequency = route_frequency.rename(columns={'route_short_name': 'route_long_name'})
            else:
                # If neither is available, just use route_id
                route_frequency = trip_counts.copy()
                route_frequency['route_long_name'] = route_frequency['route_id']

        # Sort by trip count
        route_frequency = route_frequency.sort_values('trip_count', ascending=False)

        return route_frequency
    
    def analyze_stop_activity(self):
        """Analyze the activity level at each stop."""
        if self.stop_times is None or self.stops is None:
            print("Data not loaded. Please load data first.")
            return None
        
        # Count stop occurrences in stop_times
        stop_counts = self.stop_times['stop_id'].value_counts().reset_index()
        stop_counts.columns = ['stop_id', 'activity_count']
        
        # Merge with stop information
        stop_activity = pd.merge(stop_counts, self.stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']], on='stop_id')
        
        # Sort by activity count
        stop_activity = stop_activity.sort_values('activity_count', ascending=False)
        
        return stop_activity
    
    def cluster_stops_by_location(self, n_clusters=5):
        """
        Cluster stops by geographic location to identify natural service areas.
        
        Parameters:
        -----------
        n_clusters : int
            Number of clusters to create
            
        Returns:
        --------
        stop_clusters : DataFrame
            Stops with their assigned clusters
        """
        print(f"Clustering stops into {n_clusters} service areas...")
        
        if self.stop_stats is None:
            self.stop_stats = self.analyze_stop_activity()
            
        if self.stop_stats is None:
            print("Unable to get stop activity data.")
            return None
        
        # Extract location data
        stop_locations = self.stop_stats[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'activity_count']]
        
        # Use K-means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        stop_locations['cluster'] = kmeans.fit_predict(stop_locations[['stop_lat', 'stop_lon']])
        
        # Calculate cluster statistics
        cluster_stats = stop_locations.groupby('cluster').agg({
            'stop_id': 'count',
            'activity_count': 'sum',
            'stop_lat': 'mean',
            'stop_lon': 'mean'
        }).rename(columns={'stop_id': 'stop_count'})
        
        print("Service area clustering complete.")
        
        # Store the clusters
        self.stop_clusters = stop_locations
        
        return stop_locations, cluster_stats
    
    def train_demand_prediction_model(self, test_size=0.2):
        """
        Train a machine learning model to predict demand at stops based on time and location.
        
        Parameters:
        -----------
        test_size : float
            Portion of data to use for testing
            
        Returns:
        --------
        model_performance : dict
            Model performance metrics
        """
        print("Training demand prediction model...")
        
        if self.demand_features is None:
            print("Feature data not available. Run preprocess_data() first.")
            return None
        
        # Prepare features and target
        X = self.demand_features[['stop_lat', 'stop_lon', 'departure_hour', 
                                'is_rush_hour', 'is_business_hours', 'is_night']]
        y = self.demand_features['demand']
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        
        # Preprocessing pipeline
        numeric_features = ['stop_lat', 'stop_lon', 'departure_hour']
        binary_features = ['is_rush_hour', 'is_business_hours', 'is_night']
        
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numeric_features),
                ('bin', 'passthrough', binary_features)
            ])
        
        # Create model pipeline
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('model', GradientBoostingRegressor(random_state=42))
        ])
        
        # Train model
        pipeline.fit(X_train, y_train)
        
        # Evaluate model
        y_pred = pipeline.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        print(f"Model Performance:")
        print(f"  MAE: {mae:.2f}")
        print(f"  RMSE: {rmse:.2f}")
        print(f"  R²: {r2:.4f}")
        
        # Store the model
        self.demand_model = pipeline
        self.scaler = preprocessor
        
        # Save the model
        joblib.dump(pipeline, os.path.join(self.models_dir, 'demand_prediction_model.pkl'))
        
        return {
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'model': pipeline
        }
    
    def predict_stop_demand(self, stop_id, hour):
        """
        Predict demand for a specific stop and hour.
        
        Parameters:
        -----------
        stop_id : str
            ID of the stop
        hour : int
            Hour of the day (0-23)
            
        Returns:
        --------
        predicted_demand : float
            Predicted number of trips
        """
        if self.demand_model is None:
            print("Demand prediction model not trained. Run train_demand_prediction_model() first.")
            return None
        
        if stop_id not in self.stops['stop_id'].values:
            print(f"Stop ID {stop_id} not found in data.")
            return None
        
        # Get stop location
        stop_info = self.stops[self.stops['stop_id'] == stop_id].iloc[0]
        
        # Create feature vector
        is_rush_hour = 1 if (hour >= 7 and hour <= 9) or (hour >= 16 and hour <= 18) else 0
        is_business_hours = 1 if hour >= 9 and hour <= 17 else 0
        is_night = 1 if hour >= 22 or hour <= 5 else 0
        
        features = pd.DataFrame({
            'stop_lat': [stop_info['stop_lat']],
            'stop_lon': [stop_info['stop_lon']],
            'departure_hour': [hour],
            'is_rush_hour': [is_rush_hour],
            'is_business_hours': [is_business_hours],
            'is_night': [is_night]
        })
        
        # Make prediction
        predicted_demand = self.demand_model.predict(features)[0]
        
        return max(0, round(predicted_demand, 1))  # Ensure non-negative and round
    
    def predict_system_demand(self, hour):
        """
        Predict demand across the whole system for a given hour.
        
        Parameters:
        -----------
        hour : int
            Hour of the day (0-23)
            
        Returns:
        --------
        demand_predictions : DataFrame
            Predicted demand for all stops
        """
        if self.demand_model is None:
            print("Demand prediction model not trained. Run train_demand_prediction_model() first.")
            return None
        
        print(f"Predicting system-wide demand for hour {hour}...")
        
        # Prepare features for all stops
        is_rush_hour = 1 if (hour >= 7 and hour <= 9) or (hour >= 16 and hour <= 18) else 0
        is_business_hours = 1 if hour >= 9 and hour <= 17 else 0
        is_night = 1 if hour >= 22 or hour <= 5 else 0
        
        # Create a dataframe with a row for each stop
        features = pd.DataFrame({
            'stop_id': self.stops['stop_id'],
            'stop_name': self.stops['stop_name'],
            'stop_lat': self.stops['stop_lat'],
            'stop_lon': self.stops['stop_lon'],
            'departure_hour': hour,
            'is_rush_hour': is_rush_hour,
            'is_business_hours': is_business_hours,
            'is_night': is_night
        })
        
        # Make predictions
        X_pred = features[['stop_lat', 'stop_lon', 'departure_hour', 
                          'is_rush_hour', 'is_business_hours', 'is_night']]
        features['predicted_demand'] = self.demand_model.predict(X_pred)
        
        # Ensure non-negative values
        features['predicted_demand'] = features['predicted_demand'].apply(lambda x: max(0, round(x, 1)))
        
        return features[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'predicted_demand']]
    
    def identify_underserved_stops(self):
        """
        Identify stops that are likely underserved based on predicted vs. actual demand.
        
        Returns:
        --------
        underserved_stops : DataFrame
            Stops with demand vs. service metrics
        """
        print("Identifying potentially underserved stops...")
        
        if self.demand_model is None or self.stop_stats is None:
            print("Required data or models not available.")
            return None
        
        # For each stop, predict demand during peak hours
        morning_peak = 8  # 8 AM
        evening_peak = 17  # 5 PM
        
        # Get morning and evening predictions
        morning_predictions = self.predict_system_demand(morning_peak)
        evening_predictions = self.predict_system_demand(evening_peak)
        
        # Calculate average peak demand
        peak_demand = pd.DataFrame({
            'stop_id': morning_predictions['stop_id'],
            'stop_name': morning_predictions['stop_name'],
            'stop_lat': morning_predictions['stop_lat'],
            'stop_lon': morning_predictions['stop_lon'],
            'morning_demand': morning_predictions['predicted_demand'],
            'evening_demand': evening_predictions['predicted_demand'],
            'peak_demand': (morning_predictions['predicted_demand'] + evening_predictions['predicted_demand']) / 2
        })
        
        # Merge with actual service levels
        service_comparison = pd.merge(
            peak_demand,
            self.stop_stats[['stop_id', 'activity_count']],
            on='stop_id'
        )
        
        # Calculate demand to service ratio
        service_comparison['demand_service_ratio'] = service_comparison['peak_demand'] / service_comparison['activity_count']
        
        # Identify potentially underserved stops (high demand, low service)
        service_comparison['is_underserved'] = (
            (service_comparison['peak_demand'] > service_comparison['peak_demand'].median()) & 
            (service_comparison['activity_count'] < service_comparison['activity_count'].median())
        )
        
        # Sort by demand to service ratio (higher values = more underserved)
        underserved_stops = service_comparison.sort_values('demand_service_ratio', ascending=False)
        
        return underserved_stops
    
    def create_demand_heatmap(self, hour, filename="demand_heatmap.html"):
        """
        Create an interactive heatmap of predicted demand across the system.
        
        Parameters:
        -----------
        hour : int
            Hour of the day to predict for (0-23)
        filename : str
            Name of the output HTML file
            
        Returns:
        --------
        output_path : str
            Path to the generated heatmap
        """
        print(f"Creating demand heatmap for hour {hour}...")
        
        # Get demand predictions
        predictions = self.predict_system_demand(hour)
        
        if predictions is None:
            return None
        
        # Create a map centered at the mean location of stops
        center_lat = predictions['stop_lat'].mean()
        center_lon = predictions['stop_lon'].mean()
        
        m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
        
        # Add a heatmap layer
        from folium.plugins import HeatMap
        
        # Prepare data for heatmap (lat, lon, intensity)
        heat_data = [
            [row['stop_lat'], row['stop_lon'], row['predicted_demand']] 
            for _, row in predictions.iterrows()
        ]
        
        # Add heatmap to the map
        HeatMap(heat_data).add_to(m)
        
        # Add markers for top demand locations
        top_demand = predictions.nlargest(10, 'predicted_demand')
        
        for _, row in top_demand.iterrows():
            folium.Marker(
                location=[row['stop_lat'], row['stop_lon']],
                popup=f"Stop: {row['stop_name']}<br>Predicted Demand: {row['predicted_demand']}",
                icon=folium.Icon(color='red', icon='info-sign')
            ).add_to(m)
        
        # Add time information
        time_str = f"{hour}:00"
        title_html = f'''
            <h3 align="center" style="font-size:16px"><b>Predicted Demand at {time_str}</b></h3>
        '''
        m.get_root().html.add_child(folium.Element(title_html))
        
        # Save the map
        output_path = os.path.join(self.data_dir, filename)
        m.save(output_path)
        
        print(f"Demand heatmap saved to {output_path}")
        return output_path
    
    def optimize_service_frequency(self, route_id):
        """
        Recommend optimal service frequency for a route based on predicted demand.
        
        Parameters:
        -----------
        route_id : str
            ID of the route to optimize
            
        Returns:
        --------
        recommendations : dict
            Service frequency recommendations
        """
        print(f"Optimizing service frequency for route {route_id}...")
        
        if self.demand_model is None:
            print("Demand prediction model not trained.")
            return None
        
        # Get stops served by this route
        if 'route_id' not in self.trips.columns:
            print("Route information not available.")
            return None
        
        # Get trip IDs for this route
        route_trips = self.trips[self.trips['route_id'] == route_id]['trip_id'].unique()
        
        if len(route_trips) == 0:
            print(f"No trips found for route {route_id}")
            return None
        
        # Get stops served by these trips
        route_stops = self.stop_times[self.stop_times['trip_id'].isin(route_trips)]['stop_id'].unique()
        
        # Predict demand for each hour
        hourly_demand = []
        
        for hour in range(24):
            # Get predictions for all stops on this route
            hour_predictions = self.predict_system_demand(hour)
            
            # Filter for just this route's stops
            route_demand = hour_predictions[hour_predictions['stop_id'].isin(route_stops)]
            
            # Calculate total route demand for this hour
            total_demand = route_demand['predicted_demand'].sum()
            hourly_demand.append({
                'hour': hour,
                'total_demand': total_demand,
                'stops_count': len(route_demand)
            })
        
        hourly_demand_df = pd.DataFrame(hourly_demand)
        
        # Calculate optimal frequency based on demand
        # Simple formula: 1 trip per 20 passengers predicted
        hourly_demand_df['recommended_frequency'] = (hourly_demand_df['total_demand'] / 20).round().astype(int)
        
        # Ensure minimum service levels (at least 1 trip per hour during service hours)
        hourly_demand_df.loc[
            (hourly_demand_df['hour'] >= 6) & 
            (hourly_demand_df['hour'] <= 22) & 
            (hourly_demand_df['recommended_frequency'] == 0),
            'recommended_frequency'
        ] = 1
        
        # Get route name
        route_name = "Unknown"
        if route_id in self.routes['route_id'].values:
            route_name = self.routes[self.routes['route_id'] == route_id]['route_long_name'].iloc[0]
        
        # Generate recommendations
        peak_hour = hourly_demand_df.loc[hourly_demand_df['total_demand'].idxmax()]
        
        recommendations = {
            'route_id': route_id,
            'route_name': route_name,
            'stops_served': len(route_stops),
            'peak_hour': int(peak_hour['hour']),
            'peak_demand': peak_hour['total_demand'],
            'peak_frequency': int(peak_hour['recommended_frequency']),
            'hourly_recommendations': hourly_demand_df,
            'avg_daily_trips': hourly_demand_df['recommended_frequency'].sum()
        }
        
        return recommendations
    
    def generate_ml_report(self):
        """Generate a comprehensive analysis report with ML insights."""
        print("Generating ML-enhanced transit analysis report...")
        
        # Create a reports directory
        reports_dir = os.path.join(self.data_dir, "reports")
        os.makedirs(reports_dir, exist_ok=True)
        
        # 1. Route frequency analysis
        if self.route_stats is not None:
            plt.figure(figsize=(12, 8))
            sns.barplot(x='trip_count', y='route_long_name', data=self.route_stats.head(20))
            plt.title('Top 20 Routes by Trip Frequency')
            plt.tight_layout()
            plt.savefig(os.path.join(reports_dir, 'route_frequency.png'))
            plt.close()
        
        # 2. Generate demand predictions for key hours
        key_hours = [8, 12, 17, 22]  # Morning rush, midday, evening rush, evening
        
        for hour in key_hours:
            self.create_demand_heatmap(hour, f"demand_heatmap_{hour}.html")
        
        # 3. Identify underserved stops

        underserved_stops = self.identify_underserved_stops()
        if underserved_stops is not None:
            underserved_stops.to_csv(os.path.join(reports_dir, 'underserved_stops.csv'), index=False)

            # Check if we actually have any underserved stops before plotting
            top_underserved = underserved_stops[underserved_stops['is_underserved']] if 'is_underserved' in underserved_stops.columns else pd.DataFrame()

            if not top_underserved.empty and len(top_underserved) > 0:
                # Plot top underserved stops
                plt.figure(figsize=(12, 8))
                top_underserved = top_underserved.head(15)
                sns.barplot(x='demand_service_ratio', y='stop_name', data=top_underserved)
                plt.title('Top 15 Potentially Underserved Stops')
                plt.xlabel('Demand to Service Ratio')
                plt.tight_layout()
                plt.savefig(os.path.join(reports_dir, 'underserved_stops.png'))
            else:
                # Create a figure with text explaining no underserved stops found
                plt.figure(figsize=(12, 8))
                plt.text(0.5, 0.5, "No underserved stops identified", 
                         horizontalalignment='center', verticalalignment='center',
                         fontsize=16)
                plt.axis('off')
                plt.savefig(os.path.join(reports_dir, 'underserved_stops.png'))

            plt.close()
        
        # 4. Stop clustering analysis
        stop_clusters, cluster_stats = self.cluster_stops_by_location()
        if stop_clusters is not None:
            # Create a map with clusters
            center_lat = stop_clusters['stop_lat'].mean()
            center_lon = stop_clusters['stop_lon'].mean()
            
            cluster_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)
            
            # Define colors for clusters
            colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 
                     'lightred', 'darkblue', 'darkgreen', 'cadetblue']
            
            # Add points colored by cluster
            for _, row in stop_clusters.iterrows():
                cluster_id = row['cluster']
                color = colors[cluster_id % len(colors)]
                
                folium.CircleMarker(
                    location=[row['stop_lat'], row['stop_lon']],
                    radius=5,
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=0.7,
                    popup=f"Stop: {row['stop_name']}<br>Cluster: {cluster_id}<br>Activity: {row['activity_count']}"
                ).add_to(cluster_map)
            
            # Add cluster centers
            for cluster_id, stats in cluster_stats.iterrows():
                folium.Marker(
                    location=[stats['stop_lat'], stats['stop_lon']],
                    popup=f"Cluster {cluster_id}<br>Stops: {stats['stop_count']}<br>Total Activity: {stats['activity_count']}",
                    icon=folium.Icon(color=colors[cluster_id % len(colors)], icon='info-sign')
                ).add_to(cluster_map)
            
            cluster_map.save(os.path.join(reports_dir, 'stop_clusters.html'))
        
        # 5. Optimize top routes
        if self.route_stats is not None:
            top_routes = self.route_stats.head(5)['route_id'].tolist()
            all_recommendations = []
            
            for route_id in top_routes:
                recommendations = self.optimize_service_frequency(route_id)
                if recommendations:
                    all_recommendations.append(recommendations)
            
            if all_recommendations:
                # Create plots for each route's hourly recommendations
                plt.figure(figsize=(15, 10))
                
                for i, rec in enumerate(all_recommendations):
                    plt.subplot(len(all_recommendations), 1, i+1)
                    hourly = rec['hourly_recommendations']
                    plt.bar(hourly['hour'], hourly['recommended_frequency'])
                    plt.title(f"Route: {rec['route_name']}")
                    plt.xlabel('Hour of Day')
                    plt.ylabel('Recommended Trips')
                    plt.xticks(range(0, 24, 2))
                
                plt.tight_layout()
                plt.savefig(os.path.join(reports_dir, 'route_optimization.png'))
                plt.close()
        
        # 6. Create a summary report document
        with open(os.path.join(reports_dir, 'ml_transit_analysis.txt'), 'w') as f:
            f.write("ML-Enhanced Public Transit System Analysis\n")
            f.write("=======================================\n\n")
            
            f.write("1. System Overview\n")
            f.write(f"   - Total Routes: {len(self.routes) if self.routes is not None else 'N/A'}\n")
            f.write(f"   - Total Stops: {len(self.stops) if self.stops is not None else 'N/A'}\n")
            f.write(f"   - Total Trips: {len(self.trips) if self.trips is not None else 'N/A'}\n\n")
            
            if self.route_stats is not None:
                f.write("2. Route Analysis\n")
                f.write(f"   - Most Frequent Route: {self.route_stats.iloc[0]['route_long_name']} "
                       f"({self.route_stats.iloc[0]['trip_count']} trips)\n")
                f.write(f"   - Least Frequent Route: {self.route_stats.iloc[-1]['route_long_name']} "
                       f"({self.route_stats.iloc[-1]['trip_count']} trips)\n\n")
            
            if self.demand_model is not None:
                f.write("3. Demand Prediction Model\n")
                f.write(f"   - Model Type: {type(self.demand_model[-1]).__name__}\n")

                # Instead of trying to re-evaluate the model (which would need X_test and y_test),
                # just write out general information about the model
                f.write(f"   - Algorithm: Gradient Boosting Regression\n")
                f.write(f"   - Features: Location, time of day, and contextual variables\n")
                f.write(f"   - Important Time Periods: Morning Rush (7-9 AM), Evening Rush (4-6 PM)\n\n")

            if underserved_stops is not None and not underserved_stops.empty:
                f.write("4. Service Gap Analysis\n")
                underserved_count = underserved_stops['is_underserved'].sum()
                f.write(f"   - Potentially Underserved Stops: {underserved_count}\n")
                if underserved_count > 0:
                    top_underserved = underserved_stops[underserved_stops['is_underserved']].head(5)
                    f.write("   - Top 5 Underserved Stops:\n")
                    for _, row in top_underserved.iterrows():
                        f.write(f"     * {row['stop_name']} - Demand/Service Ratio: {row['demand_service_ratio']:.2f}\n")
                f.write("\n")
            
            if cluster_stats is not None:
                f.write("5. Geographic Cluster Analysis\n")
                f.write(f"   - Number of Service Areas: {len(cluster_stats)}\n")
                for cluster_id, stats in cluster_stats.iterrows():
                    f.write(f"   - Cluster {cluster_id}: {stats['stop_count']} stops, "
                           f"{stats['activity_count']:.0f} total activity\n")
                f.write("\n")
            
            if all_recommendations:
                f.write("6. Route Optimization Recommendations\n")
                for rec in all_recommendations:
                    f.write(f"   - Route {rec['route_name']}:\n")
                    f.write(f"     * Peak Hour: {rec['peak_hour']}:00\n")
                    f.write(f"     * Peak Demand: {rec['peak_demand']:.1f}\n")
                    f.write(f"     * Recommended Peak Frequency: {rec['peak_frequency']} trips/hour\n")
                    f.write(f"     * Recommended Daily Trips: {rec['avg_daily_trips']}\n")
                f.write("\n")
            
            f.write("7. Recommendations\n")
            f.write("   - Increase service frequency on high-demand routes during peak hours\n")
            f.write("   - Address service gaps in underserved areas identified by the demand model\n")
            f.write("   - Consider route modifications to better connect high-demand areas\n")
            f.write("   - Implement real-time service adjustments based on predicted demand patterns\n")
            f.write("   - Focus additional resources on morning and evening rush hours\n")
        
        print(f"ML analysis report generated in {reports_dir}")
        return reports_dir

In [31]:
#!/usr/bin/env python3
# Machine Learning Enhanced Transit Analysis
# Example usage script

#from ml_transit_analyzer import MLTransitAnalyzer

def main():
    print("=" * 70)
    print("  MACHINE LEARNING ENHANCED PUBLIC TRANSIT ANALYSIS  ")
    print("=" * 70)
    
    # Initialize the analyzer
    analyzer = MLTransitAnalyzer()
    
    # Step 1: Download GTFS data (or use a local copy)
    print("\nStep 1: Downloading transit data")
    print("-" * 50)
    try:
        # New York MTA GTFS data for subway service
        analyzer.download_gtfs("https://api.mta.info/GTFS/nycbus/google_transit_bronx.zip")
    except Exception as e:
        print(f"Error downloading data: {e}")
        print("Using local data if available...")
    
    # Step 2: Load the data
    print("\nStep 2: Loading transit data")
    print("-" * 50)
    data_loaded = analyzer.load_data()
    
    if not data_loaded:
        print("Error: Could not load data. Exiting.")
        return
    
    # Step 3: Preprocess data for machine learning
    print("\nStep 3: Preprocessing data")
    print("-" * 50)
    analyzer.preprocess_data()
    
    # Step 4: Analyze basic transit patterns
    print("\nStep 4: Analyzing transit patterns")
    print("-" * 50)
    
    # Get route frequency
    route_freq = analyzer.analyze_route_frequency()
    print(f"Analyzed {len(route_freq)} routes")
    print("\nTop 5 most frequent routes:")
    for _, row in route_freq.head(5).iterrows():
        print(f"  - {row['route_long_name']}: {row['trip_count']} trips")
    
    # Get stop activity
    stop_activity = analyzer.analyze_stop_activity()
    print(f"\nAnalyzed {len(stop_activity)} stops")
    print("\nTop 5 busiest stops:")
    for _, row in stop_activity.head(5).iterrows():
        print(f"  - {row['stop_name']}: {row['activity_count']} arrivals/departures")
    
    # Step 5: Apply machine learning
    print("\nStep 5: Applying machine learning")
    print("-" * 50)
    
    # Train demand prediction model
    print("\nTraining demand prediction model...")
    model_perf = analyzer.train_demand_prediction_model()
    
    if model_perf:
        print(f"Model R² score: {model_perf['r2']:.4f}")
    
    # Cluster stops by location
    print("\nClustering stops by geographic location...")
    stop_clusters, cluster_stats = analyzer.cluster_stops_by_location(n_clusters=5)
    
    if cluster_stats is not None:
        print("\nIdentified transit service areas:")
        for cluster_id, stats in cluster_stats.iterrows():
            print(f"  - Area {cluster_id}: {stats['stop_count']} stops, {stats['activity_count']:.0f} total activity")
    
    # Identify underserved stops
    print("\nIdentifying potentially underserved stops...")
    underserved = analyzer.identify_underserved_stops()
    
    if underserved is not None:
        underserved_count = underserved['is_underserved'].sum()
        print(f"Identified {underserved_count} potentially underserved stops")
        
        if underserved_count > 0:
            print("\nTop 5 most underserved stops:")
            top_underserved = underserved[underserved['is_underserved']].head(5)
            for _, row in top_underserved.iterrows():
                print(f"  - {row['stop_name']}: Demand/Service Ratio = {row['demand_service_ratio']:.2f}")
    
    # Create demand heatmap for key hours
    print("\nCreating demand prediction heatmaps...")
    morning_rush = analyzer.create_demand_heatmap(8, "morning_rush_demand.html")
    midday = analyzer.create_demand_heatmap(12, "midday_demand.html")
    evening_rush = analyzer.create_demand_heatmap(17, "evening_rush_demand.html")
    
    # Optimize route frequency
    if route_freq is not None and len(route_freq) > 0:
        print("\nOptimizing service frequency for top route...")
        top_route_id = route_freq.iloc[0]['route_id']
        top_route_name = route_freq.iloc[0]['route_long_name']
        
        recommendations = analyzer.optimize_service_frequency(top_route_id)
        
        if recommendations:
            print(f"\nService optimization for Route {top_route_name}:")
            print(f"  - Peak hour: {recommendations['peak_hour']}:00")
            print(f"  - Peak demand: {recommendations['peak_demand']:.1f}")
            print(f"  - Recommended peak frequency: {recommendations['peak_frequency']} trips/hour")
            print(f"  - Recommended daily trips: {recommendations['avg_daily_trips']}")
    
    # Step 6: Generate comprehensive report
    print("\nStep 6: Generating analysis report")
    print("-" * 50)
    
    report_dir = analyzer.generate_ml_report()
    
    print("\nAnalysis complete!")
    print(f"Results are available in the {report_dir} directory")
    print("=" * 70)

if __name__ == "__main__":
    main()

  MACHINE LEARNING ENHANCED PUBLIC TRANSIT ANALYSIS  

Step 1: Downloading transit data
--------------------------------------------------
Error downloading data: File is not a zip file
Using local data if available...

Step 2: Loading transit data
--------------------------------------------------
Loading GTFS data...
Loaded 14 routes and 186 stops.

Step 3: Preprocessing data
--------------------------------------------------
Preprocessing data...
Data preprocessing complete.

Step 4: Analyzing transit patterns
--------------------------------------------------
Analyzed 12 routes

Top 5 most frequent routes:
  - Oakland Airport to Coliseum: 503 trips
  - Coliseum to Oakland Airport: 502 trips
  - Millbrae/SFIA to Antioch: 222 trips
  - Antioch to SFIA/Millbrae: 220 trips
  - Dublin/Pleasanton to Daly City: 168 trips

Analyzed 50 stops

Top 5 busiest stops:
  - Coliseum: 1917 arrivals/departures
  - Balboa Park: 1258 arrivals/departures
  - Glen Park: 1258 arrivals/departures
  - 24th

C:\Users\Richard Kagame\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Richard Kagame\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
<ipython-input-30-524ac78d7d48>:252: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_locations['cluster'] = kmeans.fit_predict(stop_locations[['stop_lat', 'stop_lon']])


Predicting system-wide demand for hour 17...
Identified 0 potentially underserved stops

Creating demand prediction heatmaps...
Creating demand heatmap for hour 8...
Predicting system-wide demand for hour 8...
Demand heatmap saved to data\morning_rush_demand.html
Creating demand heatmap for hour 12...
Predicting system-wide demand for hour 12...
Demand heatmap saved to data\midday_demand.html
Creating demand heatmap for hour 17...
Predicting system-wide demand for hour 17...
Demand heatmap saved to data\evening_rush_demand.html

Optimizing service frequency for top route...
Optimizing service frequency for route 19...
Predicting system-wide demand for hour 0...
Predicting system-wide demand for hour 1...
Predicting system-wide demand for hour 2...
Predicting system-wide demand for hour 3...
Predicting system-wide demand for hour 4...
Predicting system-wide demand for hour 5...
Predicting system-wide demand for hour 6...
Predicting system-wide demand for hour 7...
Predicting system-wide

C:\Users\Richard Kagame\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Richard Kagame\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
<ipython-input-30-524ac78d7d48>:252: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_locations['cluster'] = kmeans.fit_predict(stop_locations[['stop_lat', 'stop_lon']])



Predicting system-wide demand for hour 3...
Predicting system-wide demand for hour 4...
Predicting system-wide demand for hour 5...
Predicting system-wide demand for hour 6...
Predicting system-wide demand for hour 7...
Predicting system-wide demand for hour 8...
Predicting system-wide demand for hour 9...
Predicting system-wide demand for hour 10...
Predicting system-wide demand for hour 11...
Predicting system-wide demand for hour 12...
Predicting system-wide demand for hour 13...
Predicting system-wide demand for hour 14...
Predicting system-wide demand for hour 15...
Predicting system-wide demand for hour 16...
Predicting system-wide demand for hour 17...
Predicting system-wide demand for hour 18...
Predicting system-wide demand for hour 19...
Predicting system-wide demand for hour 20...
Predicting system-wide demand for hour 21...
Predicting system-wide demand for hour 22...
Predicting system-wide demand for hour 23...
Optimizing service frequency for route 20...
Predicting syste